In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#df = pd.read_csv('')

/kaggle/input/customer/sample_submission.csv
/kaggle/input/customer/Train.csv
/kaggle/input/customer/Test.csv


In [2]:
trainDf = pd.read_csv('/kaggle/input/customer/Train.csv')
testDf = pd.read_csv('/kaggle/input/customer/Test.csv')
sampleSubDf = pd.read_csv('/kaggle/input/customer/sample_submission.csv')

In [3]:
trainDf.sample(5)
#target column is Segmentation

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
174,458987,Female,Yes,72,Yes,Lawyer,0.0,High,2.0,Cat_6,C
43,466006,Female,Yes,49,Yes,Artist,1.0,Low,2.0,NaN,C
4039,459432,Male,Yes,84,Yes,Lawyer,0.0,Low,3.0,Cat_6,A
7498,466076,Male,No,19,No,Healthcare,8.0,Low,3.0,Cat_6,D
309,464763,Female,Yes,49,No,Engineer,NaN,Average,4.0,Cat_4,B


In [4]:
testDf.sample(5)

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
340,460108,Male,Yes,39,Yes,Artist,1.0,Average,3.0,Cat_2
1683,464728,Male,Yes,51,No,Entertainment,0.0,High,4.0,Cat_4
2019,465865,Female,No,41,Yes,Healthcare,9.0,Low,1.0,Cat_6
426,460378,Female,No,37,Yes,Doctor,1.0,Low,1.0,Cat_6
305,459996,Male,No,55,Yes,Entertainment,1.0,Low,1.0,Cat_6


In [5]:
sampleSubDf.sample(5)

,ID,Segmentation
2494,467511,A
2184,466418,A
442,460406,A
1283,463269,A
603,460976,A


In [6]:
trainDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [7]:
trainDf.describe()

,ID,Age,Work_Experience,Family_Size
count,8068.000000,8068.000000,7239.000000,7733.000000
mean,463479.214551,43.466906,2.641663,2.850123
std,2595.381232,16.711696,3.406763,1.531413
min,458982.000000,18.000000,0.000000,1.000000
25%,461240.750000,30.000000,0.000000,2.000000
50%,463472.500000,40.000000,1.000000,3.000000
75%,465744.250000,53.000000,4.000000,4.000000
max,467974.000000,89.000000,14.000000,9.000000


In [8]:
trainDf.isnull().sum()

#workexperience nanları 0 
#family size ları 1 doldur
#Graduated non knowed
#Profession non knowed
#Ever_Married preferNotSay

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

In [9]:
trainDf[trainDf['Profession'].isnull()]

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
79,466567,Female,No,49,Yes,NaN,1.0,Low,1.0,Cat_6,D
118,465827,Male,No,27,No,NaN,2.0,Low,7.0,Cat_4,D
219,465837,Male,No,62,Yes,NaN,0.0,Low,1.0,Cat_6,A
237,467252,Female,No,33,Yes,NaN,0.0,Low,4.0,NaN,D
437,461410,Male,Yes,79,No,NaN,0.0,Average,2.0,NaN,C
...,...,...,...,...,...,...,...,...,...,...,...
7743,467388,Male,No,35,NaN,NaN,0.0,Low,NaN,Cat_6,D
7870,462301,Female,No,27,No,NaN,12.0,Low,3.0,Cat_6,D
7899,464548,Female,Yes,47,No,NaN,1.0,Low,1.0,Cat_4,A
7935,464977,Female,Yes,66,No,NaN,1.0,Average,2.0,Cat_4,B


In [10]:
trainDf[trainDf['Ever_Married'].isnull()]

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
24,461021,Female,NaN,58,No,Executive,1.0,Average,3.0,Cat_3,B
108,466466,Female,NaN,19,No,Healthcare,6.0,Low,5.0,Cat_3,D
201,466065,Male,NaN,19,No,Healthcare,9.0,Low,3.0,Cat_3,D
213,460516,Female,NaN,85,No,Lawyer,0.0,High,1.0,Cat_3,C
272,464841,Male,NaN,19,No,Entertainment,0.0,High,3.0,Cat_4,D
...,...,...,...,...,...,...,...,...,...,...,...
7756,465987,Male,NaN,20,No,Healthcare,1.0,Low,3.0,Cat_2,D
7775,462989,Male,NaN,32,Yes,Healthcare,1.0,Low,1.0,Cat_6,D
8011,466026,Female,NaN,49,No,Entertainment,0.0,Low,1.0,Cat_3,A
8030,459082,Male,NaN,45,Yes,Artist,1.0,Low,NaN,Cat_6,A


In [11]:
#workexperience nanları 0 
#family size ları 1 doldur
#Graduated non knowed
#Profession non knowed
#Ever_Married preferNotSay

In [12]:
trainDf.loc[(trainDf['Ever_Married'].isnull()),'Ever_Married']= 'noncommittal'
trainDf.loc[(trainDf['Work_Experience'].isnull()),'Work_Experience']= 0
trainDf.loc[(trainDf['Family_Size'].isnull()),'Family_Size']= 1
trainDf.loc[(trainDf['Graduated'].isnull()),'Graduated']= 'noncommittal'
trainDf.loc[(trainDf['Profession'].isnull()),'Profession']= 'noncommittal'

In [13]:
trainDf.loc[(trainDf['Ever_Married'].isnull())]

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation


In [14]:
trainDf['Ever_Married'].value_counts()

Yes             4643
No              3285
noncommittal     140
Name: Ever_Married, dtype: int64

In [15]:
trainDf.isnull().sum()

ID                  0
Gender              0
Ever_Married        0
Age                 0
Graduated           0
Profession          0
Work_Experience     0
Spending_Score      0
Family_Size         0
Var_1              76
Segmentation        0
dtype: int64

In [16]:
labelEncoder = LabelEncoder()
#Gender           8068 non-null   object 
#2   Ever_Married     7928 non-null   object 
#4   Graduated        7990 non-null   object 
#5   Profession       7944 non-null   object 
#7   Spending_Score   8068 non-null   object 
trainDf['Ever_Married'] = labelEncoder.fit_transform(trainDf['Ever_Married'])
trainDf['Graduated'] = labelEncoder.fit_transform(trainDf['Graduated'])
trainDf['Profession'] = labelEncoder.fit_transform(trainDf['Profession'])
trainDf['Spending_Score'] = labelEncoder.fit_transform(trainDf['Spending_Score'])
trainDf['Gender'] = labelEncoder.fit_transform(trainDf['Gender'])

trainDf['Segmentation'] = labelEncoder.fit_transform(trainDf['Segmentation'])

trainDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   int64  
 2   Ever_Married     8068 non-null   int64  
 3   Age              8068 non-null   int64  
 4   Graduated        8068 non-null   int64  
 5   Profession       8068 non-null   int64  
 6   Work_Experience  8068 non-null   float64
 7   Spending_Score   8068 non-null   int64  
 8   Family_Size      8068 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   int64  
dtypes: float64(2), int64(8), object(1)
memory usage: 693.5+ KB


In [17]:
feature_columns = ['Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession', 'Work_Experience', 'Spending_Score', 'Family_Size']
target_column = ['Segmentation']

In [18]:
from sklearn.preprocessing import normalize,scale

In [19]:
data_scaled = scale(trainDf[feature_columns])
data_scaled

array([[ 0.90916397, -1.16877275, -1.28462344, ..., -0.41210249,
         0.754462  ,  0.7945189 ],
       [-1.0999116 ,  0.74665657, -0.32715082, ..., -0.71285715,
        -1.60301259,  0.14682985],
       [-1.0999116 ,  0.74665657,  1.40826831, ..., -0.41210249,
         0.754462  , -1.14854823],
       ...,
       [-1.0999116 , -1.16877275, -0.62636101, ..., -0.41210249,
         0.754462  , -1.14854823],
       [-1.0999116 , -1.16877275, -0.98541324, ..., -0.41210249,
         0.754462  ,  0.7945189 ],
       [ 0.90916397,  0.74665657, -0.38699286, ..., -0.71285715,
        -1.60301259,  0.14682985]])

In [20]:
data_scaled.max()
data_scaled.min()

-1.60301259307534

In [21]:
scale([110,20])

array([ 1., -1.])

In [22]:
data_normal = normalize(trainDf[feature_columns])

In [23]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = scaler.fit_transform(trainDf[feature_columns])
scaled

array([[1.        , 0.        , 0.05633803, ..., 0.07142857, 1.        ,
        0.375     ],
       [0.        , 0.5       , 0.28169014, ..., 0.        , 0.        ,
        0.25      ],
       [0.        , 0.5       , 0.69014085, ..., 0.07142857, 1.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.21126761, ..., 0.07142857, 1.        ,
        0.        ],
       [0.        , 0.        , 0.12676056, ..., 0.07142857, 1.        ,
        0.375     ],
       [1.        , 0.5       , 0.26760563, ..., 0.        , 0.        ,
        0.25      ]])

In [24]:
x_train,x_test,y_train,y_test = train_test_split(scaled, trainDf[target_column], test_size=0.2)

In [25]:
y_train.sample(10)

,Segmentation
6819,1
4224,0
3761,3
5222,1
2082,2
5127,0
1143,1
6264,3
1308,3
3084,0


In [26]:
randomFor = RandomForestRegressor(n_estimators = 100, random_state = 0)
fitted = randomFor.fit(x_train,y_train)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [27]:
y_pred = randomFor.predict(x_test)

In [28]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.8301104579022796
Mean Squared Error: 1.152901862328862
Root Mean Squared Error: 1.0737326773125897


In [29]:
#Lets try with drop na cols

In [30]:
trainDfDp = trainDf.copy()

In [31]:
trainDfDp.dropna(inplace=True)

In [32]:
trainDfDp.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64

In [33]:
from sklearn.preprocessing import normalize,scale
data_scaled = scale(trainDfDp[feature_columns])
data_scaled
scale([110,20])

array([ 1., -1.])

In [34]:
x_train_dp,x_test_dp,y_train_dp,y_test_dp = train_test_split(data_scaled, trainDfDp[target_column], test_size=0.2)

In [35]:
randomFor_dp = RandomForestRegressor(n_estimators = 100, random_state = 0)
fitted_dp = randomFor_dp.fit(x_train_dp,y_train_dp)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [36]:
y_pred_dp = randomFor_dp.predict(x_test_dp)

In [37]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_dp, y_pred_dp))
print('Mean Squared Error:', metrics.mean_squared_error(y_test_dp, y_pred_dp))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test_dp, y_pred_dp)))

Mean Absolute Error: 0.8186009309119066
Mean Squared Error: 1.1280733256061837
Root Mean Squared Error: 1.0621079632533519
